In [1]:
import pandas as pd
import numpy as np

from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk

In [2]:
analysis_2019 = pd.read_csv('rdata/data_2019.csv')

In [3]:
analysis_2019.head()

,Unnamed: 0,question,summary,tags,user_id,no_of_votes,no_of_answers,date,clean_text
0,0,multiple json objects into a list,"{""Id"": 1,""product"": ""Mobile"",""price"": 32889,""d...","['python', 'json']",diveinsky,0.0,1.0,2019-08-02,multiple json object list
1,1,How to implement like button concept in viewpa...,I have a viewpager which displays images and b...,"['android', 'android-viewpager']",Marat Zangiev,1.0,1.0,2019-08-02,implement like button concept viewpager
2,2,Is it a good practice using Observable with as...,I am using angular 2 common http that return a...,"['javascript', 'angular', 'typescript', 'promi...",Ashwin J Chhetri,48.0,2.0,2019-08-02,good practice use observable async await
3,3,UBSan: Store to misaligned address; what is th...,"I've been running some code under UBSan, and f...",['ubsan'],Flamefire,0.0,1.0,2019-08-02,ubsan store misalign address problem care
4,4,Django-Filter Form Displaying All Filters?,I've just started working with Django Filter. ...,"['django-filter', 'django-filters']",udeep shrestha,0.0,2.0,2019-08-02,django filter form display filter


In [4]:
analysis_2019.shape

(2517347, 9)

In [5]:
analysis_2019['clean_text'].isna().sum()

16

In [6]:
analysis_2019 = analysis_2019.dropna(subset=['clean_text'])

In [7]:
analysis_2019['clean_text'] = analysis_2019['clean_text'].str.split(' ')

In [8]:
def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=5, no_above=0.5):
  print('Building dictionary...')
  dictionary = Dictionary(docs)
  stopwords = nltk_stopwords().union(additional_stopwords)
  stopword_ids = map(dictionary.token2id.get, stopwords)
  dictionary.filter_tokens(stopword_ids)
  dictionary.compactify()
  dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
  dictionary.compactify()

  print('Building corpus...')
  corpus = [dictionary.doc2bow(doc) for doc in docs]

  return dictionary, corpus

In [9]:
dictionary, corpus = prep_corpus(analysis_2019['clean_text'])

Building dictionary...
Building corpus...


In [10]:
MmCorpus.serialize('models/2019_corpus.mm', corpus)
dictionary.save('models/2019_dict.dict')

In [11]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=25)
                                      
lda.save('models/2019_lda.model')

CPU times: user 28min 19s, sys: 12.9 s, total: 28min 32s
Wall time: 11min 15s


In [12]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [13]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(vis_data)